In [6]:
import pandas as pd 
import numpy as np
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [7]:
# Read in the data 
df = pd.read_csv("familyData.csv")
categoricalFeatures = ["employmentStatus", "ownership", "participation"]

numericalFeatures = ['liquidWealth', 'laborIncome', 'costPerPerson', 'totalExpense',
                        'investmentAmount', 'annuityIRA', 'wealthWithoutHomeEquity', 
                        'wealthWithHomeEquity', 'HomeEquity','stockInvestmentRatio']

features = categoricalFeatures + numericalFeatures
df_features = df[features]
# Standardize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_features[numericalFeatures] = scaler.fit_transform(df_features[numericalFeatures])

In [10]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(df_features)
df['cluster'] = kmeans.labels_

In [29]:
df.groupby('cluster').education.value_counts(normalize=True)

cluster  education   
0        college         0.542893
         postGraduate    0.283920
         highSchool      0.148735
         middleSchool    0.024452
1        postGraduate    0.701752
         college         0.275705
         highSchool      0.021894
         middleSchool    0.000649
Name: education, dtype: float64

In [30]:
df.groupby('cluster').industry.value_counts(normalize=True)

cluster  industry   
0        noneFinance    0.972848
         finance        0.027152
1        noneFinance    0.951022
         finance        0.048978
Name: industry, dtype: float64

In [32]:
df.groupby('cluster')[numericalFeatures].mean()

,liquidWealth,laborIncome,costPerPerson,totalExpense,investmentAmount,annuityIRA,wealthWithoutHomeEquity,wealthWithHomeEquity,HomeEquity,stockInvestmentRatio
cluster,,,,,,,,,,
0,7.872066,42.112789,13.909241,31.205714,2.877291,9.764158,39.068785,77.456276,38.387460,0.024998
1,83.256439,105.512246,25.191750,58.652423,198.646854,200.150851,834.132308,1097.293745,263.161406,0.264649
